In [92]:
import os
import cv2
from PIL import Image
import moviepy.video.fx.all as vfx 
import re
from gtts import gTTS
from langchain_ollama import OllamaLLM
from transformers import BlipProcessor, BlipForConditionalGeneration
from ultralytics import YOLO
from moviepy.editor import VideoFileClip, AudioFileClip,concatenate_videoclips
import moviepy.audio.fx.all as vfx 
from langchain.prompts import ChatPromptTemplate

In [2]:
video_path = "C:/Users/HP/Downloads/3195366-uhd_3840_2160_25fps.mp4"
audio_output = "video_comentry.mp3"
final_video_output = "final_output_with_audio.mp4"


In [ ]:
model=YOLO('yolov8n.pt')
processor=BlipProcessor.from_pretrained('Salesforce/blip-image-captioning-base')
caption_model=BlipForConditionalGeneration.from_pretrained('Salesforce/blip-image-captioning-base')

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Error while downloading from https://cdn-lfs.hf.co/repos/f1/cb/f1cbe4cfb2a267026632ce513d5918162e03df2ee28456145ba8a1d25cf39aad/90f4c493dfc6f4c2152c8bfcfd52ee141055feb835eea2dceb8b9231e5dbf17c?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1753539933&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc1MzUzOTkzM319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy9mMS9jYi9mMWNiZTRjZmIyYTI2NzAyNjYzMmNlNTEzZDU5MTgxNjJlMDNkZjJlZTI4NDU2MTQ1YmE4YTFkMjVjZjM5YWFkLzkwZjRjNDkzZGZjNmY0YzIxNTJjOGJmY2ZkNTJlZTE0MTA1NWZlYjgzNWVlYTJkY2ViOGI5MjMxZTVkYmYxN2M%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qIn1dfQ__&Signature=toerhbefCJIX86n6G6DxOtVNboSEnxjJgmJ3Mtcvt6NcNf6EakHhRpGsr1j45IZ2k1xgPhu2R7AT1%7ECv%7Etazbdqqec1e2vdfdCnq29qiJMjr1%7Es48wcYo5I76m78fX8vuDH4uRP%7Eh7FgL4fxyOStKxyoqlf9ehFwEB3UP-rqa5jUkARUO8KPCIHEiTdyusS5b2A07NSDnbwNH7rA1ejEzPpCZ1gm8QVtCQaLzh9JiqtTeJL4BFzIQfjx-rD5Cv5RZMB

In [5]:
llm=OllamaLLM(model='llama3')

In [6]:
cap=cv2.VideoCapture(video_path)

In [8]:
dialogs=[]
count=0
total_frames=int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print("\n[INFO] Starting frame processing...")


[INFO] Starting frame processing...


In [9]:
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Resize and convert frame to RGB
    height, width = frame.shape[:2]
    frame = cv2.resize(frame, (width // 6, height // 6))
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_image = Image.fromarray(frame_rgb)

    # Generate caption
    inputs = processor(images=frame_image, return_tensors='pt')
    outputs = caption_model.generate(**inputs)
    caption = processor.decode(outputs[0], skip_special_tokens=True)

    if caption not in dialogs:
        dialogs.append(caption)

    count += 1
    print(f"Processed frame {count}/{total_frames} - Caption: {caption}")

cap.release()
cv2.destroyAllWindows()

Processed frame 1/449 - Caption: a soccer goal with a soccer ball in the goal
Processed frame 2/449 - Caption: a soccer goal with a soccer ball in the goal
Processed frame 3/449 - Caption: a soccer goal with a soccer ball in the goal
Processed frame 4/449 - Caption: a soccer goal with a soccer ball in the goal
Processed frame 5/449 - Caption: a soccer goal with a soccer ball in the goal
Processed frame 6/449 - Caption: a soccer goal with a soccer ball in the goal
Processed frame 7/449 - Caption: a soccer goal with a soccer ball in the goal
Processed frame 8/449 - Caption: a soccer goal with a man in the background
Processed frame 9/449 - Caption: a soccer goal with a man in the background
Processed frame 10/449 - Caption: a soccer goal with a man in the background
Processed frame 11/449 - Caption: a soccer goal with a man in the background
Processed frame 12/449 - Caption: a soccer goal with a man in the background
Processed frame 13/449 - Caption: a soccer goal with a man in the backg

In [10]:
full_dialog='.'.join(dialogs)
print("\n[INFO] Generating voiceover script using LLM...")


[INFO] Generating voiceover script using LLM...


In [66]:
prompt = ChatPromptTemplate.from_messages([
    ("system", 
     "You are a passionate soccer commentator narrating a highlight video. The following text contains scene captions from a soccer match. Write a short, vivid, emotionally charged voiceover — like you're calling a live ESPN highlight. Use energetic storytelling, avoid lists, and keep it under 30 seconds."),
    ("human", "{text}")
])



"""prompt = ChatPromptTemplate.from_messages([
    ("system", 
     "You are a passionate soccer commentator narrating a highlight video. The following text contains automatically generated scene captions from a soccer match video. Based on them, write an exciting, energetic voiceover as if you're describing the scenes live on air. Use natural storytelling — no lists, no analysis. Just pure commentary with emotion, momentum, and drama. Make it cinematic and flowing, like a highlight reel on ESPN."),
    ("human", "{text}")
])
"""

'prompt = ChatPromptTemplate.from_messages([\n    ("system", \n     "You are a passionate soccer commentator narrating a highlight video. The following text contains automatically generated scene captions from a soccer match video. Based on them, write an exciting, energetic voiceover as if you\'re describing the scenes live on air. Use natural storytelling — no lists, no analysis. Just pure commentary with emotion, momentum, and drama. Make it cinematic and flowing, like a highlight reel on ESPN."),\n    ("human", "{text}")\n])\n'

In [67]:
formatted_prompt=prompt.format_messages(text=full_dialog)

In [68]:
response=llm.invoke(formatted_prompt)
print(response)

"FOLKS, WE'VE GOT A GOAL OF THE SEASON RIGHT HERE! The Red Devil, number 10, has just written his name across the scoreboard in bold letters! He receives the pass, beats two defenders with a mesmerizing move, and UNLEASHES A BEAUTY FROM JUST OUTSIDE THE BOX! The ball dips and swerves, leaving the keeper grasping at air as it finds its way into the net. The crowd ERUPTS in ecstasy as our hero celebrates with his teammates. This is what soccer dreams are made of, folks! The Red Devil has given his team the lead, and we're witnessing pure MAGIC on this beautiful grass!"


In [69]:
def remove_brackets(text):
    cleaned=re.sub(r'\([^)]*\)', '', text) 
    cleaned=re.sub(r'\([^)]*\)', '', cleaned) 
    cleaned=re.sub(r'\s+',' ',cleaned).strip()
    return cleaned

In [70]:
response=remove_brackets(response)

In [72]:
while not os.path.exists(audio_output):
    pass
print("\n[INFO] Stitching audio with video using moviepy...")


[INFO] Stitching audio with video using moviepy...


In [73]:
video_clip=VideoFileClip(video_path)
audio_clip=AudioFileClip(audio_output)

In [86]:
video_duration = video_clip.duration
audio_duration = audio_clip.duration

In [88]:
speed_factor=video_duration/(audio_duration*2)
loop_count=int(video_duration//audio_duration)+1

In [96]:
looped_clip=[video_clip]*loop_count
looped_video=video_clip.loop(duration=audio_clip.duration)

In [97]:
final_clip = looped_video.set_audio(audio_clip)

In [98]:
final_clip.write_videofile(final_video_output, codec='libx264', audio_codec='aac')


Moviepy - Building video final_output_with_audio.mp4.
MoviePy - Writing audio in final_output_with_audioTEMP_MPY_wvf_snd.mp4


chunk:   0%|          | 0/987 [00:00<?, ?it/s, now=None]

MoviePy - Done.
Moviepy - Writing video final_output_with_audio.mp4



Moviepy - Done !
Moviepy - video ready final_output_with_audio.mp4


In [99]:
print("\n[INFO] Opening final video...")
os.startfile(final_video_output)


[INFO] Opening final video...
